<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Risk Parity & Budgeting with Python

### Risk Allocation & Risk Parity

&copy; Dr. Yves J. Hilpisch | The Python Quants GmbH

http://tpq.io | [@dyjh](http://twitter.com/dyjh) | [team@tpq.io](mailto:team@tpq.io) 

## Imports and Data

In [ ]:
from pylab import plt
from mvp_portfolio import *
plt.style.use('seaborn')
np.set_printoptions(suppress=True)
%config InlineBackend.figure_format = 'svg'

In [ ]:
url = 'universe.csv'

In [ ]:
data = pd.read_csv(url, index_col=0, parse_dates=True).dropna()
data = data.loc['2019-1-1':]

In [ ]:
rets = np.log(data / data.shift(1))

In [ ]:
noa = len(data.columns)
noa

## Equally Weighted Portfolio

In [ ]:
phi = np.array(noa * [1 / noa])
phi

In [ ]:
plt.pie(phi, labels=data.columns, autopct='%1.1f%%')
plt.title('Equally Weighted Portfolio');

In [ ]:
cov = rets.cov() * 252

In [ ]:
cov

In [ ]:
vol = portfolio_volatility(phi, rets)
vol

In [ ]:
vol / noa  # "average" risk contribution

In [ ]:
np.dot(cov, phi)

In [ ]:
mvols = np.dot(cov, phi) / vol  # marginal volatilities
mvols

In [ ]:
rc = mvols * phi  # risk contributions (%-points)
rc

In [ ]:
rc.sum()

In [ ]:
rrc = rc / rc.sum()  # relative risk contributions (%)
rrc

In [ ]:
rrc.sum()

In [ ]:
plt.pie(rrc, labels=data.columns, autopct='%1.1f%%')
plt.title('Relative Risk Contributions');

In [ ]:
sum(rc / rc.sum())  # checking for sum of relative risk contributions

## Risk Parity Portfolios

In [ ]:
from scipy.optimize import minimize

In [ ]:
def rel_risk_contributions(weights, rets=rets):
    vol = portfolio_volatility(weights, rets)
    cov = rets.cov()
    mvols = np.dot(cov, weights) / vol
    rc = mvols * weights
    rrc = rc / rc.sum()
    return rrc

In [ ]:
rrc = rel_risk_contributions(phi)
rrc

In [ ]:
plt.pie(rrc, labels=data.columns, autopct='%1.1f%%')
plt.title('Equally Weighted Portfolio');

In [ ]:
def mse_risk_contributions(weights, target, rets=rets):
    rc = rel_risk_contributions(weights, rets)
    mse = ((rc - target) ** 2).mean()
    return mse * 100

In [ ]:
mse_risk_contributions(phi, phi)

In [ ]:
bnds = noa * [(0, 1),]

In [ ]:
cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}

In [ ]:
target = noa * [1 / noa,]  # risk parity as target
target = [0.2, 0.2, 0.2, 0.4]  # example risk budget as target
target

In [ ]:
opt = minimize(lambda w: mse_risk_contributions(w, target=target),
               phi, bounds=bnds, constraints=cons)

In [ ]:
opt

In [ ]:
phi_ = opt['x']

In [ ]:
plt.pie(phi_, labels=data.columns, autopct='%1.1f%%')
plt.title('Optimal Portfolio Weights');

In [ ]:
rrc = rel_risk_contributions(opt['x'])
rrc

In [ ]:
plt.pie(rrc, labels=data.columns, autopct='%1.1f%%')
plt.title('Relative Risk Contributions');

In [ ]:
portfolio_volatility(opt['x'], rets)

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="30%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">team@tpq.io</a>